In [97]:
import pandas as pd
import numpy as np

Pour reference:
http://www.fao.org/faostat/fr/#data

# Question 1
donnez le résultat de votre calcul pour l'année 2013, ainsi que pour la dernière année  disponible au jour où vous effectuez ce projet.

In [98]:
# Loading csv file
population = pd.read_csv("population_world_2013.csv")


In [99]:
# Rebranding significant columns
population.columns = ["xx1", "source", "id", "pays", "code", "element", "xx3", "type", "year", "yearbis", "unit", "value", "xx6", "xx7"]
#population.set_index('pays', inplace=True)
population.loc[population["pays"]=="France"]

,xx1,source,id,pays,code,element,xx3,type,year,yearbis,unit,value,xx6,xx7
58,FBS,Bilans Alimentaires,68,France,511,Population totale,2501,Population,2013,2013,1000 personnes,64291,NaN,Donnée officielle


In [112]:
population.shape

(175, 14)

In [100]:
# Pivoting and filtering on the value column
data = population.pivot_table(index=["pays", "year"], values=["value"])
data.sum()*1000

value    8413993000
dtype: int64

In [101]:
# Aggregation_chine
data.loc[["Chine, continentale", "Chine, Taiwan Province de", "Chine - RAS de Macao", "Chine - RAS de Hong-Kong"], 'value'].sum()

1416667

On obtient ainsi une estimation de la population mondiale pour l'annee 2013 a plus de 8,4 milliards.

Ce chiffre est tres largement superieur aux estimations de l'ONU pour l'annee 2013 qui se situent a 7 milliards pour l'annee 2013. Par ailleurs pour l'annee 2019, l'estimation actuelle est de 7,7 milliards (source Ined https://www.ined.fr/fr/tout-savoir-population/chiffres/tous-les-pays-du-monde/)

Notre calcul est donc bien superieur a l'estimation. En parcourant la liste des noms de pays, on remarque que la Chine apparait a plusieurs reprises selon differentes declinaisons. 
Ici l'unite des chiffres utilises est une unite par millier de personnes (marge d'erreur max de 2%) 
La source des donnees pour la Chine est issue d'aggregat de donnees et pese plus de 16% de la population mondiale. 
Certaines donnees sont manquantes (URSS, Coree du Nord).


In [102]:
# Aggregation_chine
data.loc["Chine", "value"]

year
2013    1416667
Name: value, dtype: int64

Les donnees concernant la population chinoise sont en doublon. Aussi il ne faut retenir que la valeur globale. On retrouve alors un calcul plus proche de l'estimation qui est a 7 milliards.

# Question 2
Identifiez ces redondances, en donnant votre réponse sous forme de formule mathématique (pas besoin de coder ici). C'est une équation à 3 termes de type \(a_1 + a2 + [...] = b_1 + b_2 + [...] = c_1 + c_2 + [...]\) ) faisant intervenir chacune des 11 quantités données ci dessus. Illustrez cette équation avec l'exemple du blé en France. Pour avoir un indice, cliquez sur "Définitions et Standards" sur la page de téléchargement des données.
* Production (Production)
* Importations (Import Quantity)
* Exportations (Export Quantity)
* Variation de stock (Stock Variation)
* Disponibilité intérieure (Domestic supply)
* Semences (Seed)
* Pertes (Waste)
* Nourriture (Food) , aussi appelée Disponibilité alimentaire (Food Supply)
* Aliments pour animaux (Feed)
* Traitement (Processing)
* Autres utilisations (Other uses)


In [103]:
vegetal = pd.read_csv("veg.csv")
vegetal.columns = ["xx1", "source", "id", "pays", "code", "element", "xx3", "type", "year", "yearbis", "unit", "value", "xx6", "xx7"]

# Global
# vegetal

# Focus on France
vegetal.loc[vegetal["pays"]=="France"].head()

,xx1,source,id,pays,code,element,xx3,type,year,yearbis,unit,value,xx6,xx7
30989,FBS,Food Balance Sheets,68,France,5511,Production,2511,Wheat and products,2013,2013,1000 tonnes,38614.0,S,Standardized data
30990,FBS,Food Balance Sheets,68,France,5611,Import Quantity,2511,Wheat and products,2013,2013,1000 tonnes,2055.0,S,Standardized data
30991,FBS,Food Balance Sheets,68,France,5072,Stock Variation,2511,Wheat and products,2013,2013,1000 tonnes,1131.0,S,Standardized data
30992,FBS,Food Balance Sheets,68,France,5911,Export Quantity,2511,Wheat and products,2013,2013,1000 tonnes,21502.0,S,Standardized data
30993,FBS,Food Balance Sheets,68,France,5301,Domestic supply quantity,2511,Wheat and products,2013,2013,1000 tonnes,20298.0,S,Standardized data


Au regard des chiffres de la France pour le ble, on intuite les relations suivantes:
* Domestic Supply = Production + Import Quantity - Export Quantity + Stock variation
* Domestic Supply = Food + Feed + Seed + Losses + Other Uses + Processing

In [104]:
vegetal.set_index(['pays', 'element', 'type'], inplace=True)
vegetal.shape

(96715, 11)

In [105]:
# Hypothesis check
production = vegetal.loc['France', 'Production', 'Wheat and products'].value
import_quantity = vegetal.loc['France', 'Import Quantity', 'Wheat and products'].value
stock_variation = vegetal.loc['France', 'Stock Variation', 'Wheat and products'].value
export_quantity = vegetal.loc['France', 'Export Quantity', 'Wheat and products'].value

food = vegetal.loc['France', 'Food', 'Wheat and products'].value
feed = vegetal.loc['France', 'Feed', 'Wheat and products'].value
seed = vegetal.loc['France', 'Seed', 'Wheat and products'].value
losses = vegetal.loc['France', 'Losses', 'Wheat and products'].value
other_uses = vegetal.loc['France', 'Other uses', 'Wheat and products'].value
processing = vegetal.loc['France', 'Processing', 'Wheat and products'].value

domestic_supply = vegetal.loc['France', 'Domestic supply quantity', 'Wheat and products'].value

In [106]:
# First check
domestic_supply - (food + feed + seed + losses + other_uses + processing)

0.0

In [107]:
# Second check
domestic_supply - (production + import_quantity - export_quantity + stock_variation)

0.0

On en conclut que:
Domestic supply = feed + seed + losses + other_uses + processing = production + import_quantity - export_quantity + stock_variation


# Question 3
Calculez (pour chaque pays et chaque produit) la disponibilité alimentaire en kcal puis en kg de protéines.

Vous ferez cela à partir de ces informations :

* Population de chaque pays
* Disponibilité alimentaire ("Food Supply" en anglais) donnée pour chaque produit et pour chaque pays en kcal/personne/jour.
* Disponibilité alimentaire en protéines ("Protein supply quantity" en anglais) donnée pour chaque produit et pour chaque pays en g/personne/jour.

In [138]:
# Loading data regarging animal based food
animal = pd.read_csv("ani.csv")
animal.columns = ["xx1", "source", "id", "pays", "code", "element", "xx3", "type", "year", "yearbis", "unit", "value", "xx6", "xx7"]

animal.head()

,xx1,source,id,pays,code,element,xx3,type,year,yearbis,unit,value,xx6,xx7
0,FBS,Food Balance Sheets,2,Afghanistan,5511,Production,2731,Bovine Meat,2013,2013,1000 tonnes,134.00,S,Standardized data
1,FBS,Food Balance Sheets,2,Afghanistan,5611,Import Quantity,2731,Bovine Meat,2013,2013,1000 tonnes,6.00,S,Standardized data
2,FBS,Food Balance Sheets,2,Afghanistan,5301,Domestic supply quantity,2731,Bovine Meat,2013,2013,1000 tonnes,140.00,S,Standardized data
3,FBS,Food Balance Sheets,2,Afghanistan,5142,Food,2731,Bovine Meat,2013,2013,1000 tonnes,140.00,S,Standardized data
4,FBS,Food Balance Sheets,2,Afghanistan,645,Food supply quantity (kg/capita/yr),2731,Bovine Meat,2013,2013,kg,4.59,Fc,Calculated data


In [139]:
#animal relation stats
animal.shape

(33445, 14)

Apres avoir charge les donnees alimentaires d'origine vegetale et animale, on 'append' les deux relations pour ainsi ajouter les lignes des deux relations. On fait de meme avec les donnees de la population

In [140]:
food = vegetal.append(animal)
food = food.append(population)

In [141]:
food.shape

(130335, 14)

In [142]:
food = food.pivot_table(
    index=["id","pays","code","element","year"],
    columns = ["type"], values=["value"], 
    aggfunc=sum
)


In [136]:
food.head(20)

value  \
type                                                            Aquatic Animals, Others   
id pays        code element                                year                           
1  Armenia     645  Food supply quantity (kg/capita/yr)    2013                     0.0   
               664  Food supply (kcal/capita/day)          2013                     0.0   
               674  Protein supply quantity (g/capita/day) 2013                     0.0   
               5072 Stock Variation                        2013                     NaN   
               5123 Losses                                 2013                     NaN   
               5131 Processing                             2013                     NaN   
               5142 Food                                   2013                     0.0   
               5154 Other uses                             2013                     0.0   
               5301 Domestic supply quantity               2013                     0.0   
               5511 Production                             2013                     NaN   
               5521 Feed                                   2013                     NaN   
               5527 Seed                                   2013                     NaN   
               5611 Import Quantity                        2013                     0.0   
               5911 Export Quantity                        2013                     NaN   
   Arménie     511  Population totale                      2013                     NaN   
2  Afghanistan 511  Population totale                      2013                     NaN   
               645  Food supply quantity (kg/capita/yr)    2013                     NaN   
               664  Food supply (kcal/capita/day)          2013                     NaN   
               674  Protein supply quantity (g/capita/day) 2013                     NaN   
               5123 Losses                                 2013                     NaN   

                                                                                \
type                                                            Aquatic Plants   
id pays        code element                                year                  
1  Armenia     645  Food supply quantity (kg/capita/yr)    2013            0.0   
               664  Food supply (kcal/capita/day)          2013            0.0   
               674  Protein supply quantity (g/capita/day) 2013            0.0   
               5072 Stock Variation                        2013            NaN   
               5123 Losses                                 2013            NaN   
               5131 Processing                             2013            NaN   
               5142 Food                                   2013            0.0   
               5154 Other uses                             2013            0.0   
               5301 Domestic supply quantity               2013            0.0   
               5511 Production                             2013            0.0   
               5521 Feed                                   2013            NaN   
               5527 Seed                                   2013            NaN   
               5611 Import Quantity                        2013            0.0   
               5911 Export Quantity                        2013            0.0   
   Arménie     511  Population totale                      2013            NaN   
2  Afghanistan 511  Population totale                      2013            NaN   
               645  Food supply quantity (kg/capita/yr)    2013            NaN   
               664  Food supply (kcal/capita/day)          2013            NaN   
               674  Protein supply quantity (g/capita/day) 2013            NaN   
               5123 Losses                                 2013            NaN   

                                                                             \
type                  